In [ ]:
from groq import Groq
import json
from tqdm import tqdm
import os

client = Groq()

In [ ]:
import json
from tqdm import tqdm
with open('/home/nkama/LLM_and_RAG_Course/LLM_and_RAG-/notebooks/documents-with-ids.json', 'rt') as f_in:
    documents = json.load(f_in)

In [33]:
import getpass
def _get_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_get_env("GROQ_API_KEY")

In [11]:
def generate_test_data(doc):
    prompt_template = """
    You emulate a student who's taking our course.
    Formulate 5 questions this student might ask based on a FAQ record. The record
    should contain the answer to the questions, and the questions should be complete and not too short.
    If possible, use as fewer words as possible from the record.

    The record:

    section: {section}
    question: {question}
    answer: {text}

    Provide the output in parsable JSON without using code blocks:

    ["question1", "question2", ..., "question5"]
    """.strip()
    prompt = prompt_template.format(**doc)

    response = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content":prompt,
            }
        ],
        model="llama3-8b-8192"
    )

    json_response = response.choices[0].message.content
    return json_response

results = {}
for doc in tqdm(documents):
    doc_id = doc['id']
    if doc_id in results:
        continue

    questions = generate_test_data(doc)
    results[doc_id] = questions

100%|██████████| 948/948 [39:44<00:00,  2.52s/it]


In [13]:
import pickle
from tqdm import tqdm

# Pickle the results dictionary
with open('generated_test_data.pkl', 'wb') as f:
    pickle.dump(results, f)

print("Data has been pickled and saved to 'results.pkl'")


Data has been pickled and saved to 'results.pkl'


In [14]:
with open('generated_test_data.pkl', 'rb') as f:
    loaded_results = pickle.load(f)

print("Data has been loaded from 'results.pkl'")


Data has been loaded from 'results.pkl'


In [29]:
import json

parsed_results = {}

for doc_id, json_questions in loaded_results.items():
    try:
        # If escape sequences are an issue, correct them
        questions = json_questions.replace('\\', '\\\\')

        # Parse the JSON string
        parsed_results[doc_id] = json.loads(questions)
    except json.JSONDecodeError as e:
        print(f"Failed to decode JSON for doc_id {doc_id}: {e}")
        print(f"Problematic JSON string: {json_questions}")


Failed to decode JSON for doc_id c02e79ef: Expecting value: line 1 column 1 (char 0)
Problematic JSON string: Here are 5 questions a student might ask based on the FAQ record in parsable JSON format:

[
  "What is the exact date and start time of the course?",
  "Will the course start with an introduction or a lecture?",
  "How can I access the course schedule?",
  "Is it necessary to register before the course starts?",
  "How can I join the course communities on Telegram and DataTalks.Club's Slack?"
]
Failed to decode JSON for doc_id 1f6520ca: Expecting value: line 1 column 1 (char 0)
Problematic JSON string: Here are 5 questions a student might ask based on the FAQ record:

[
    "What are the necessary requirements to take this course",
    "Could you provide a list of courses I need to complete before taking this one",
    "What skills or knowledge do I need to have before I start this course",
    "Are there any specific prerequisites I need to meet to enroll in this course",
   

In [19]:
doc_index = {d['id']: d for d in documents}

In [32]:
final_results = []

for doc_id, questions in parsed_results.items():
    course = doc_index[doc_id]['course']
    for q in questions:
        final_results.append((q, course, doc_id))

In [35]:
import pandas as pd

In [36]:
df = pd.DataFrame(final_results, columns=['question', 'course', 'document'])

In [37]:
df.to_csv('ground-truth-data.csv', index=False)

In [38]:
!head ground-truth-data.csv

question,course,document
When does the course begin?,data-engineering-zoomcamp,c02e79ef
How can I get the course schedule?,data-engineering-zoomcamp,c02e79ef
What is the link for course registration?,data-engineering-zoomcamp,c02e79ef
How can I receive course announcements?,data-engineering-zoomcamp,c02e79ef
Where do I join the Slack channel?,data-engineering-zoomcamp,c02e79ef
Where can I find the prerequisites for this course?,data-engineering-zoomcamp,1f6520ca
How do I check the prerequisites for this course?,data-engineering-zoomcamp,1f6520ca
Where are the course prerequisites listed?,data-engineering-zoomcamp,1f6520ca
What are the requirements for joining this course?,data-engineering-zoomcamp,1f6520ca
